# Модуль re

В Python для работы с регулярными выражениями используется модуль **re**.

Основные функции модуля **re**:

* `match` - ищет последовательность в начале строки
* `search` - ищет первое совпадение с шаблоном
* `findall` - ищет все совпадения с шаблоном. Возвращает результирующие строки в виде списка
* `finditer` - ищет все совпадения с шаблоном. Возвращает итератор
* `compile` - компилирует регулярное выражение. К этому объекту затем можно применять все перечисленные функции
* `fullmatch` - вся строка должна соответствовать описанному регулярному выражению

Кроме функций для поиска совпадений, в модуле есть такие функции:

* `re.sub` - для замены в строках
* `re.split` - для разделения строки на части

## Объект `Match`

В модуле **re** несколько функций возвращают объект `Match`, если было найдено совпадение:

* `search`
* `match`
* `finditer` возвращает итератор с объектами `Match`

Пример объекта Match:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'Host (\S+) in vlan (\d+) .* port (\S+) and port (\S+)', log)

match

<re.Match object; span=(47, 124), match='Host f03a.b216.7ad7 in vlan 10 is flapping betwee>

Вывод в 3 строке просто отображает информацию об объекте. Поэтому не стоит полагаться на то, что отображается в части match, так как отображаемая строка обрезается по фиксированному количеству знаков.

### `group`

Метод `group` возвращает подстроку, которая совпала с выражением или с выражением в группе.

Если метод вызывается без аргументов, отображается вся подстрока:

In [ ]:
match.group()

'Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

Аналогичный вывод возвращает группа 0:

In [ ]:
match.group(0)

'Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

Другие номера отображают только содержимое соответствующей группы:

In [ ]:
print(match.group(1))
print(match.group(2))
print(match.group(3))
print(match.group(4))

f03a.b216.7ad7
10
Gi0/5
Gi0/15


Если вызвать метод `group` с номером группы, который больше, чем количество существующих групп, возникнет ошибка:

In [ ]:
match.group(5)

IndexError: no such group

Если вызвать метод с несколькими номерами групп, результатом будет кортеж со строками, которые соответствуют совпадениям:

In [ ]:
match.group(1, 2, 3)

('f03a.b216.7ad7', '10', 'Gi0/5')

В группу может ничего не попасть, тогда ей будет соответствовать пустая строка:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'Host (\S+) in vlan (\D*)', log)

match.group(2)

''

Если группа описывает часть шаблона и совпадений было несколько, метод отобразит последнее совпадение:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'Host (\w{4}\.)+', log)

match.group(1)

'b216.'

Такой вывод получился из-за того, что выражение в скобках описывает 4 буквы или цифры, точка и после этого стоит плюс. Соответственно, сначала с выражением в скобках совпала первая часть MAC-адреса, потом вторая. Но запоминается и возвращается только последнее выражение.

Если в выражении использовались именованные группы, методу `group` можно передать имя группы и получить соответствующую подстроку:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'Host (?P<mac>\S+) '
    r'in vlan (?P<vlan>\d+) .* '
    r'port (?P<int1>\S+) '
    r'and port (?P<int2>\S+)',
    log)

print(match.group('mac'))
print(match.group('int2'))    

f03a.b216.7ad7
Gi0/15


При этом группы доступны и по номеру:

In [ ]:
print(match.group(3))
print(match.group(4))

Gi0/5
Gi0/15


### `groups`

Метод `groups` возвращает кортеж со строками, в котором элементы - это те подстроки, которые попали в соответствующие группы:

In [ ]:
log = 'Jun  3 14:39:05.941: %SW_MATM-4-MACFLAP_NOTIF: Host f03a.b216.7ad7 in vlan 10 is flapping between port Gi0/5 and port Gi0/15'

match = re.search(r'Host (\S+) '
    r'in vlan (\d+) .* '
    r'port (\S+) '
    r'and port (\S+)',
    log)
    
match.groups()

('f03a.b216.7ad7', '10', 'Gi0/5', 'Gi0/15')

У метода `groups` есть опциональный параметр - `default`. Он срабатывает в ситуации, когда все, что попадает в группу, опционально.

Например, при такой строке, совпадение будет и в первой группе, и во второй:

In [ ]:
line = '100     aab1.a1a1.a5d3    FastEthernet0/1'

match = re.search(r'(\d+) +(\w+)?', line)

match.groups()

('100', 'aab1')

Если же в строке нет ничего после пробела, в группу ничего не попадет. Но совпадение будет, так как в регулярном выражении описано, что группа опциональна:

In [ ]:
line = '100     '

match = re.search(r'(\d+) +(\w+)?', line)

match.groups()

Соответственно, для второй группы значением будет `None`.

Если передать методу groups значение `default`, оно будет возвращаться вместо `None`:

In [ ]:
line = '100     '

match = re.search(r'(\d+) +(\w+)?', line)

print(match.groups(default=0))
print(match.groups(default))